## Paralelización

In [22]:
import random
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as opt
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from joblib import Parallel, delayed 
from IPython.display import display, Markdown

### Simulación de portafolios

In [23]:
tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'SPOT', 'SBUX', 'JPM', 'AXP', 'MCD', 'KO', 
           'NFLX', 'CMG', 'CP', 'WMT', 'V', 'GLD', 'BLK', 'PG', 'JNJ', 'TMUS', 'MA', 'BX', 'LULU', 'DPZ', 'BAC', 
           'FDX', 'DIS', 'GE', 'HSY', 'HP', 'COST', 'HD', 'K', 'ADBE', 'CSCO', 'T', 'F', 'NKE', 'CVX', 'XOM', 'PYPL', 
           'PEP', 'PFE', 'MRNA', 'RL', 'AZN', 'BABA', 'VZ', 'WBD', 'HSBC', 'UBER']

rf = 0.04413
f_inicial='2020-01-01'
f_final='2024-11-24'

In [ ]:
def simulate_portfolio_opt(i, tickers, rf):
    # Selección aleatoria del tamaño de la muestra y de los tickers
    sample_size = random.randint(5, 15)
    random_tickers = random.sample(tickers, sample_size)

    # Descarga de datos
    datos1 = yf.download(random_tickers, f_inicial, f_final)['Adj Close']
    rt = datos1.pct_change().dropna()

    # Calcular retornos y matriz de covarianza
    mu = (rt.mean() * 252).values  # retornos esperados
    sigma = rt.cov().values  # Matriz de covarianza
    n_assets = len(mu)

    # Función para minimizar (-Sharpe Ratio)
    def neg_sharpe_ratio(w, mu, sigma, rf):
        port_return = np.dot(w.T, mu)  # Rendimiento esperado del portafolio
        port_vol = np.sqrt(np.dot(w.T, np.dot(sigma, w))) * np.sqrt(252)  # Volatilidad del portafolio
        sharpe_ratio = (port_return - rf) / port_vol
        return -sharpe_ratio  # Negativo porque queremos maximizar

    # Restricciones: Suma de pesos = 1
    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })

    # Límites: Pesos entre 0 y 1 (no posiciones cortas)
    bounds = tuple((0, 1) for _ in range(n_assets))

    # Pesos iniciales (distribuidos uniformemente)
    w0 = np.array([1 / n_assets] * n_assets)

    # Optimización
    result = opt.minimize(neg_sharpe_ratio, w0, args=(mu, sigma, rf), method='SLSQP', bounds=bounds, constraints=constraints)

    # Resultados
    optimal_weights = result.x
    optimal_weights[optimal_weights < 0.000001] = 0  # Eliminar pesos numéricamente cercanos a cero

    # Cálculo del retorno, volatilidad y sharpe del portafolio
    ret = np.dot(optimal_weights.T, mu)  # Retorno esperado del portafolio
    risk = np.sqrt(np.dot(optimal_weights.T, np.dot(sigma, optimal_weights))) * np.sqrt(252)  # Volatilidad del portafolio
    sharpe = (ret - rf) / risk  # Sharpe ratio

    return ret, risk, sharpe, optimal_weights, random_tickers

# Número de simulaciones
n_simulations = 100

# Ejecutar simulaciones en paralelo
results = Parallel(n_jobs=-1)(delayed(simulate_portfolio_opt)(i, tickers, rf) for i in range(n_simulations))

# Extraer resultados
retornos_sci = [result[0] for result in results]
volatilidades_sci = [result[1] for result in results]
sharpe_ratios_sci = [result[2] for result in results]
pesos_sci = [result[3] for result in results]
selected_tickers = [result[4] for result in results]  # Guardar los tickers seleccionados

[*********************100%***********************]  5 of 5 completedd
[*********************100%***********************]  6 of 6 completedd
[*********************100%***********************]  8 of 8 completedd
[*********************100%***********************]  7 of 7 completedd
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  9 of 9 completeded
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  11 of 11 completed
[*********************100%***********************]  13 of 13 completed
[*********************100%***********************]  8 of 8 completedd
[**********

### Guardar métricas en un DataFrame

In [25]:
metricas_df_scipy = pd.DataFrame({
    'rendimiento': retornos_sci,
    'volatilidad': volatilidades_sci,
    'sharpe': sharpe_ratios_sci
})
metricas_df_scipy.head()

,rendimiento,volatilidad,sharpe
0,0.287332,0.282758,0.860108
1,0.475979,0.289837,1.489973
2,0.232482,0.274887,0.685200
3,0.311913,0.267954,0.999359
4,0.241334,0.340518,0.579130


### Guardar los pesos y acciones en un DataFrame

In [26]:
dataframes_dict = {}
for i in range(n_simulations):
    dataframes_dict[f"Simulación {i+1}"] = pd.DataFrame(pesos_sci[i], index=selected_tickers[i], columns=['w']).T

### Mostrar los pesos de cada activo en una simulación

In [27]:
# Elegir el número de simulación cambiando el número
dataframes_dict['Simulación 1']

,CVX,GOOGL,JNJ,NKE,HSBC,F,AZN,CMG,BAC,DPZ,NFLX,BABA
w,0.005246,0.0,0.0,0.522963,0.077273,0.0,0.0,0.123058,0.062769,0.0,0.208691,0.0


### Mostrar todos los resultados

In [28]:
for i in range(n_simulations):
    display(Markdown(f"### Simulación {i+1}"))
    display(dataframes_dict[f"Simulación {i+1}"])

### Simulación 1

,CVX,GOOGL,JNJ,NKE,HSBC,F,AZN,CMG,BAC,DPZ,NFLX,BABA
w,0.005246,0.0,0.0,0.522963,0.077273,0.0,0.0,0.123058,0.062769,0.0,0.208691,0.0


### Simulación 2

,MA,NVDA,HSBC,XOM,COST,WMT,FDX,META,JNJ,CP,VZ,V,RL,MRNA
w,0.215651,0.0,0.0,0.0,0.0,0.0,0.0,0.065091,0.39833,0.0,0.0,0.0,0.197685,0.123243


### Simulación 3

...

,DIS,V,FDX,BLK,NFLX,JPM,HD,GOOGL,VZ,AAPL,MRNA,RL,SBUX
w,0.417378,0.0,0.0,0.066349,0.0,0.007457,0.108213,0.154793,0.167699,0.078111,0.0,0.0,0.0


### Simulación 99

,UBER,F,CSCO,KO,BX,JPM,MCD,SBUX,PFE
w,0.847048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.152952


### Simulación 100

,VZ,META,MA,HSBC,MRNA,DPZ,GOOGL,SBUX,AZN,AAPL,BAC
w,0.537794,0.0,0.0,0.078936,0.015268,0.079505,0.0,0.122898,0.165599,0.0,0.0
